In [5]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from tqdm import tqdm  # 進度條
import matplotlib.pyplot as plt
from PIL import Image

# 設定設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 設定圖片路徑
dataset_path = "crop"
batch_size = 32
IMG_SIZE = 64

# 影像轉換 (預處理)
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 標準化 -1 到 1
])

# 建立資料集
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))  # 80% 訓練，20% 測試
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# 建立 DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 顯示類別對應
print(f"類別對應: {dataset.class_to_idx}")

# 建立 CNN 模型
class CoffeeBeanCNN(nn.Module):
    def __init__(self):
        super(CoffeeBeanCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 類別 (good, bad)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 初始化模型
model = CoffeeBeanCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型 (加入進度條)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    # tqdm 進度條
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        
        # 更新進度條資訊
        progress_bar.set_postfix(loss=loss.item(), acc=100 * correct / total)

    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss:.4f}, Accuracy: {train_acc:.2f}%")

# 儲存模型
torch.save(model.state_dict(), "coffee_bean_cnn.pth")
print("✅ 模型已儲存!")

# 測試模型 (加入進度條)
model.eval()
correct = 0
total = 0
progress_bar = tqdm(test_loader, desc="Testing", leave=True)

with torch.no_grad():
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        # 更新進度條資訊
        progress_bar.set_postfix(acc=100 * correct / total)

test_acc = 100 * correct / total
print(f"📊 測試準確度: {test_acc:.2f}%")


類別對應: {'bad': 0, 'good': 1}


Epoch 1/10: 100%|██████████| 2/2 [00:00<00:00, 17.17it/s, acc=48.8, loss=0.692]


Epoch 1/10 - Loss: 1.3944, Accuracy: 48.78%


Epoch 2/10: 100%|██████████| 2/2 [00:00<00:00, 18.27it/s, acc=61, loss=0.806]


Epoch 2/10 - Loss: 1.5076, Accuracy: 60.98%


...

Epoch 9/10 - Loss: 1.2761, Accuracy: 60.98%


Epoch 10/10: 100%|██████████| 2/2 [00:00<00:00, 32.98it/s, acc=61, loss=0.583]


Epoch 10/10 - Loss: 1.1884, Accuracy: 60.98%
✅ 模型已儲存!


Testing: 100%|██████████| 1/1 [00:00<00:00, 83.74it/s, acc=72.7]

📊 測試準確度: 72.73%
